In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error as mse
import lightgbm as lgb
import time
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
import sys
print(sys.path)
from tool import tool
from datetime import timedelta


In [ ]:
data = pd.read_csv('./data2/数据预处理10.25.csv',parse_dates=['date'])
# data = data[~(data.Temp == 0.0)].reset_index(drop=True)
data.shape

In [ ]:
fe_rolling_columns = ['Spd','Temp']
fe_lag_columns = ['Spd','Temp']
fe_diff_columns = ['Spd','Temp']
fe_div_columns = ['Temp']
# fe_groupby_columns = ['Spd','Temp']
# fe_max_min_columns = ["Spd","Temp"]

# temp + spd:  max-min in a day

In [ ]:
# def fe_max_min(data, time_col,time_varying_cols):
#     """
#     构造最大最小值
#     :param df: DataFrame
#     :param time_col: time column
#     :param time_varying_cols: time varying columns
#     :param lag: lag list
#     :return: DataFrame lag
#     """
#     df = data.copy()
#     result = pd.DataFrame({time_col:df[time_col].values},index=df.index)
#     add_fetures = []
#     for column in time_varying_cols:
#         name = f'{column}_max_min'
#         add_fetures.append(name)
#         df[name] = df.groupby(["Day"])[column].transform(lambda x: (x.max() - x.min()))
#     return result.merge(df[[time_col,] + add_fetures], on=time_col, how='left')[add_fetures]

In [ ]:
# data["max-min"] = data.groupby(["Day"])["Temp"].transform(lambda x: (x.max() - x.min()))
# data

In [ ]:
# df_max_min = fe_max_min(data,"date",fe_max_min_columns)

# rolling


In [ ]:
df_rolling = tool.fe_rolling_stat(data, time_col='date', time_varying_cols=fe_rolling_columns, window_size=[3,6,24])

# lag

In [ ]:
df_lag = tool.fe_lag(data, 'date', fe_lag_columns, [1,2,3,24,48,72])  
df_lag.shape

# 一阶差分和差除

* diff：不同lag步长的一阶差分

In [ ]:
def fe_diff(data, time_col,time_varying_cols):
    """
    构造差分特征:这里和diff函数的效果不一样(diff: lag0 - lag1,lag0 - lag2, lag0 - lag3)
    lag0 - lag1 
    lag1 - lag2
    lag2 - lag3
    :param df: dataFrame
    :param time_col: time column
    :param time_varying_cols: time varying columns
    :param lag: lag list
    :return: dataFrame lag
    """
    df = data.copy()
    result = pd.DataFrame({time_col:df[time_col].values},index=df.index)
    add_fetures = []
    for column in time_varying_cols:
        name = f'{column}_onehour_diff'          # 这个名字根据情况自己取
        add_fetures.append(name)
        df[name] = df[column] - df[column].shift(1)

        name = f'{column}_twohour_diff'  # 这个名字根据情况自己取
        add_fetures.append(name)
        df[name] = df[column].shift(1) - df[column].shift(2)
 
        name = f'{column}_threehour_diff'   # 这个名字根据情况自己取
        add_fetures.append(name)
        df[name] = df[column].shift(2) - df[column].shift(3)    # lag2 - lag3
        
    
    return result.merge(df[[time_col,] + add_fetures], on=time_col, how='left')[add_fetures]

def fe_div(data, time_col,time_varying_cols):    
    """
    构造一阶差分特征,使用不同的lag
    注意不要有0值
    :param df: dataFrame
    :param time_col: time column
    :param time_varying_cols: time varying columns
    :param lag: lag list
    :return: dataFrame lag
    """
    df = data.copy()
    result = pd.DataFrame({time_col:df[time_col].values},index=df.index)
    add_fetures = []
    for column in time_varying_cols:
        name = f'{column}_onehour_div'
        add_fetures.append(name)
        df[name] = np.divide(df[column], df[column].shift(1))

        name = f'{column}_twohour_div'
        add_fetures.append(name)
        df[name] = np.divide(df[column].shift(1), df[column].shift(2))

        name = f'{column}_threehour_div'
        add_fetures.append(name)
        df[name] = np.divide(df[column].shift(2), df[column].shift(3))
    
    
    return  result.merge(df[[time_col,] + add_fetures], on=time_col, how='left')[add_fetures]

In [ ]:
df_diff = fe_diff(data, 'date', fe_diff_columns)
df_div = fe_div(data, 'date', fe_div_columns)
df_diff.shape, df_div.shape

# 根据一阶构造二阶

In [ ]:
df_div

In [ ]:
# 注意这里新增了date列，后面合并的时候记得删除
df_diff["date"] = data["date"]
df_div["date"] = data["date"]
fe_diff_2jie_columns = [x for x in list(df_diff.columns) if x != "date"]
fe_div_2jie_columns = [x for x in list(df_div.columns) if x != "date"]

# 2阶差分差除
df_diff_2jie = tool.fe_diff(df_diff, 'date', fe_diff_2jie_columns,[1])  # lag0 - lag1
df_div_2jie = tool.fe_div(df_div, 'date', fe_div_2jie_columns,[1])
df_diff_2jie.shape,df_div_2jie.shape

In [ ]:
# df_diff1 = tool.fe_diff(data, 'date', fe_diff_columns,[1,2,3])  
# df_div1 = tool.fe_div(data, 'date', fe_div_columns,[1,2,3])

# groupby

- 训练集
- 注意：避免数据泄露，训练集、测试集分开构造特征

In [ ]:
# valid_days = 10
# delta = timedelta(days=valid_days)
# time_col = 'date'
# test_time_split = data[time_col].max() - delta
# test_idx = data.loc[data[time_col] > test_time_split].index
# train_idx = data.loc[data[time_col] <= test_time_split].index

In [ ]:
# test_idx.shape,train_idx.shape

In [ ]:
# # 增加月份信息
# data['Month'] = data[time_col].dt.month
# data.shape

In [ ]:
# # 一年内的每个月的temp、spd同比的统计值
# df_groupby_train = tool.fe_groupby(data.loc[train_idx], 'date', ['Hour','Month'], fe_diff_columns)
# df_groupby_test = tool.fe_groupby(data.loc[test_idx], 'date', ['Hour','Month'], fe_diff_columns)

In [ ]:
# df_groupby = pd.concat([df_groupby_train,df_groupby_test],axis=0).reset_index(drop=True)
# df_groupby.shape

In [ ]:

def fe_groupby(data, time_col, by, time_varying_cols):
    '''
    一年内的temp、spd同比的统计值(不是一个月)
    '''
    df = data.copy()
    result = pd.DataFrame({time_col:df[time_col].values},index=df.index)
    add_fetures = []
    for val in time_varying_cols:
        for op in ['mean','std','median','max','min','skew']:
            name = f'{val}__{by[0]}__{op}'
            add_fetures.append(name)
            df[name] = df.groupby(by)[val].transform(op)
    
    return result.merge(df[[time_col,] + add_fetures], on=time_col, how='left')[add_fetures]

In [ ]:
# df_groupby_train = fe_groupby(data.loc[train_idx], 'date', ['Hour'], fe_diff_columns)
# df_groupby_test = fe_groupby(data.loc[test_idx], 'date', ['Hour'], fe_diff_columns)

In [ ]:
# df_groupby_2 = pd.concat([df_groupby_train,df_groupby_test],axis=0).reset_index(drop=True)
# df_groupby_2.shape

# 熵

In [ ]:
# data_entropy = pd.concat([data[["date","Day","Temp","Spd"]],df_lag_R],axis=1)
# data_entropy

In [ ]:
# def fe_qbins(data,time_col,use_cols,n_bins):
#     '''
#     :param data: dataFrame
#     :param time_col: time column
#     :param use_cols: list
#     :param nbins: int
#     '''
#     df = data.copy()
#     result = pd.DataFrame({time_col:df[time_col].values},index=df.index)
#     add_features = []
#     for col in use_cols:
#         name = f"{col}_{n_bins}"
#         add_features.append(name)
#         df[name] = pd.qcut(df[col],n_bins)
#     return result.merge(df[[time_col,] + add_features], on=time_col, how='left')[add_features]

# def fe_bins(data,time_col,use_cols,n_bins):
#     '''
#     :param data: dataFrame
#     :param time_col: time column
#     :param use_cols: list
#     :param nbins: int
#     '''
#     df = data.copy()
#     result = pd.DataFrame({time_col:df[time_col].values},index=df.index)
#     add_features = []
#     for col in use_cols:
#         name = f"{col}_{n_bins}"
#         add_features.append(name)
#         df[name] = pd.cut(df[col],n_bins)
#     return result.merge(df[[time_col,] + add_features], on=time_col, how='left')[add_features]

In [ ]:
# df_bins1 = fe_qbins(data_entropy,"date",["Temp","Spd"],10)  # 等距分箱
# df_bins2 = fe_bins(data_entropy,"date",["Radiation_lag_240"],20)  # 等频分箱

In [ ]:
# from sklearn.preprocessing import OrdinalEncoder
# ordinalencoder = OrdinalEncoder()

# for col in df_bins1.columns:
#     df_bins1[col] = ordinalencoder.fit_transform(df_bins1[col].values.reshape(-1,1)) 
# for col in df_bins2.columns:
#     df_bins2[col] = ordinalencoder.fit_transform(df_bins2[col].values.reshape(-1,1)) 

In [ ]:
# data_entropy = pd.concat([data_entropy,df_bins1,df_bins2],axis=1)
# data_entropy

In [ ]:
# fes_entropy_cols = ["Temp_10","Spd_10","Radiation_lag_240_20"]

In [ ]:
def fe_entropy(data,time_col,use_cols):
    """
    """
    df = data.copy()
    result = pd.DataFrame({time_col:df[time_col].values},index=df.index)
    add_fetures = []
    for col in use_cols:
        name = f'{col}_entropy'
        add_fetures.append(name)
        S = -(df.groupby(["Day"])[col].value_counts()/24) * np.log(df.groupby(["Day"])[col].value_counts()/24)
        S.index.names = ["Day",f"{col}_value"]
        S = S.reset_index()
        S.columns = ["Day",f'{col}',name]
        df_tmp = S.groupby(["Day"])[name].sum()
        df = pd.merge(df,df_tmp,on="Day",how="left")
    return result.merge(df[[time_col] + add_fetures], on=time_col, how='left')[add_fetures]
    

In [ ]:
# df_entropy = fe_entropy(data_entropy,"date",use_cols = fes_entropy_cols)

## 合并Temp+Spd的熵

In [ ]:
# # 两者的大小不一样，先归一化后再合并
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# trans_df_entropy = ["Temp_10_entropy","Spd_10_entropy"]
# for fea in trans_df_entropy:
#     df_entropy[fea] = scaler.fit_transform(df_entropy[fea].values.reshape(-1,1))

# df_entropy["Temp_Spd_entropy"] = df_entropy['Temp_10_entropy'] * 0.5 + df_entropy['Spd_10_entropy'] * 0.5

# feature combination

In [ ]:
# df_all = tool.feature_combination([data,df_rolling, df_lag,df_lag_R,df_diff.drop("date",axis=1),df_div.drop("date",axis=1),\
#     df_diff_2jie,df_div_2jie,df_groupby,df_groupby_2,df_entropy,df_max_min])

df_all = tool.feature_combination([data,df_rolling, df_lag,df_diff.drop("date",axis=1),df_div.drop("date",axis=1),\
    df_diff_2jie,df_div_2jie])

df_all.shape

In [ ]:
df_all["date"]

In [ ]:
# df_1012.to_csv('./new_fes_1012_2.0.csv',index=False)

## 对rooling特征继续构造差分

In [ ]:
fe_rolling_diff_columns = ['Spd__3h__mean','Temp__3h__mean','Spd__6h__mean','Temp__6h__mean','Spd__24h__mean','Temp__24h__mean']

In [ ]:
df_rolling_diff = fe_diff(df_all, "date", fe_rolling_diff_columns)
df_rolling_diff.shape

In [ ]:
df_all = tool.feature_combination([df_all, df_rolling_diff])

In [ ]:
df_all["date"]

# 正余弦编码

In [ ]:
hours_in_day = 24
# hour
df_all['sin_hour'] = np.sin(2 * np.pi * df_all["Hour"]/hours_in_day)
df_all['cos_hour'] = np.cos(2 * np.pi * df_all["Hour"]/hours_in_day)


In [ ]:
df_all["date"]

In [ ]:
# df_all.to_csv("./data2/特征工程10.25.csv",index=False)

## 对熵特征构造一阶差分

In [ ]:
# fe_entropy_diff_cols = ["Temp_10_entropy","Spd_10_entropy","Temp_Spd_entropy"]
# df_entropy_diff =  tool.fe_diff(df_all,"date",fe_entropy_diff_cols, [1])
# df_all = tool.feature_combination([df_all, df_entropy_diff])

In [ ]:
# df_entropy_diff.columns

In [ ]:
# df_all['Radiation'].fillna(0.0,inplace=True)
# df_all.shape

In [ ]:
# new_added_feas = ["date",'Temp_10_entropy_diff_1', 'Spd_10_entropy_diff_1','Temp_Spd_entropy_diff_1',"Spd_max_min","Temp_max_min"]
# df_all[new_added_feas].to_csv('./new_feas_1017.csv',index=False)